<a href="https://colab.research.google.com/github/Dagger77/amzn_recsys/blob/main/eda_notebooks/Amazon_Reviews_2018_Feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##IMPORT

In [ ]:
import os

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
sns.set(style="white")

In [ ]:
from sklearn.preprocessing import LabelEncoder
import torch
device = 0 if torch.cuda.is_available() else -1
from datasets import Dataset
from transformers import pipeline


import tensorflow as tf
import tensorflow_hub as hub
from tqdm import tqdm
import time

tqdm.pandas()

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


##FUNCTIONS

In [ ]:
def embed(input, model):
  if isinstance(input, str):
    input = [input]
  return model(input)

In [ ]:
def parallel_embed(df, feature, embedding_model, batch_size=128):
    #function to create description embeddings
    start_time = time.time()

    batches = [df[feature][i:i + batch_size] for i in range(0, len(df), batch_size)]
    embeddings = []

    for batch in tqdm(batches, desc="Embedding Batches"):
        batch_embeddings = embed(batch.tolist(), embedding_model).numpy()
        embeddings.extend(batch_embeddings)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Time taken to embed {feature}: {elapsed_time:.2f} seconds")
    return embeddings

In [ ]:
def analyze_sentiment_tf(batch):
    summaries = [summary.decode('utf-8') for summary in batch.numpy()]
    results = sentiment_task(summaries)
    sentiments = [result['label'] for result in results]
    return sentiments

##LOAD

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Velotix HA/amazon_review_clean.parquet')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 510312 entries, 0 to 551158
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   userName     510312 non-null  object        
 1   verified     510312 non-null  bool          
 2   itemName     510312 non-null  object        
 3   description  510312 non-null  object        
 4   brand        510312 non-null  object        
 5   feature      510312 non-null  object        
 6   category     510312 non-null  object        
 7   price        510312 non-null  float64       
 8   rating       510312 non-null  float64       
 9   reviewTime   510312 non-null  datetime64[ns]
 10  summary      510312 non-null  object        
 11  reviewText   510312 non-null  object        
 12  vote         510312 non-null  int64         
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 51.1+ MB


##FEATURES

###Drop redundant

In [ ]:
columns_to_drop = ['verified', 'feature', 'reviewTime', 'reviewText', 'vote']
df = df.drop(columns=columns_to_drop)

###Encode categorical

In [ ]:
# Label encoding for brand and category
df['brand_encoded'] = LabelEncoder().fit_transform(df['brand'])
df['category_encoded'] = LabelEncoder().fit_transform(df['category'])

In [ ]:
# Map the userName column to the user_id column
user_to_id_map = {name: idx for idx, name in enumerate(df['userName'].unique())}
df['user_id'] = df['userName'].map(user_to_id_map)

# Map the itemName column to the item_id column
items_to_id_map = {name: idx for idx, name in enumerate(df['itemName'].unique())}
df['item_id'] = df['itemName'].map(items_to_id_map)

###Embed Item description

In [ ]:
#Unversal Sentence Encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embedding_model = hub.load(module_url)

In [ ]:
df['desc_emb'] = None
df['desc_emb'] = parallel_embed(df, 'description', embedding_model)

Embedding Batches: 100%|██████████| 3987/3987 [07:31<00:00,  8.84it/s]

Time taken to embed description: 451.13 seconds


###Detect sentiment for the Review Summary

In [ ]:
#Feature can be further used as implicit feedback.
#finetuned for sentiment task distilBERT
chosen_model = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'
sentiment_task = pipeline(task="sentiment-analysis", model=chosen_model, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
#toy test
sentiment_task('Better than a blanket to cover your seat, not worth $30')

[{'label': 'NEGATIVE', 'score': 0.9987327456474304}]

In [ ]:
dataset = Dataset.from_pandas(df[['summary']])

tf_dataset = tf.data.Dataset.from_tensor_slices(dataset['summary'])

batch_size = 128
tf_dataset = tf_dataset.batch(batch_size)

sentiment_results = []
for batch in tqdm(tf_dataset, desc="Processing Batches"):
    sentiments = analyze_sentiment_tf(batch)
    sentiment_results.extend(sentiments)

df['sentiment'] = sentiment_results

Processing Batches: 100%|██████████| 3987/3987 [56:03<00:00,  1.19it/s]


In [ ]:
df['sentiment'] = df['sentiment'].apply(lambda x: x.numpy() if isinstance(x, tf.Tensor) else x)

##SAVE DATA

In [ ]:
path = '/content/drive/MyDrive/Velotix HA/'
file_path = os.path.join(path, 'amazon_processed.parquet')

df.to_parquet(file_path)

print(f"File saved to {file_path}")

File saved to /content/drive/MyDrive/Velotix HA/amazon_processed.parquet
